In [12]:
import boto3
import pandas as pd

from shared_utilities import helpers

In [8]:
if not boto3.DEFAULT_SESSION:
    ! aws sso login --profile Stellaralgo-DataScienceAdmin
    session = boto3.setup_default_session(profile_name='Stellaralgo-DataScienceAdmin')

In [15]:
CNXN = helpers.get_redshift_connection("prod-app", "stlrmls")

In [24]:
# call the stored proc to get data for the retention model
cur = CNXN.cursor()
cur.execute(f"CALL ds.getretentionmodeldata(13, 2010, 2021, 'rkcursor')")

# create a named cursor based on the cursor name passed in above
named_cursor = CNXN.cursor('rkcursor')
data = named_cursor.fetchall()

# load the data and columns into a data frame
cols = [row[0] for row in named_cursor.description]
df = pd.DataFrame(data=data, columns=cols)

CNXN.commit()

In [27]:
df["isnextyear_buyer"].value_counts()

0    2632
Name: isnextyear_buyer, dtype: int64

In [22]:
df_parq = pd.read_parquet("/Users/ryankazmerik/Downloads/dynamo.parquet")

In [23]:
df_parq["isnextyear_buyer"].value_counts()

0    2632
Name: isnextyear_buyer, dtype: int64